In [ ]:
import pandas as pd
import pathlib
import numpy as np
from stockstats import StockDataFrame
from bokeh.plotting import figure, show, output_notebook

# This tells Jupyter not to autoreload code changes on the notebook from disk
%load_ext autoreload
%autoreload 0

# this tells Bokeh not to autosave plots to files
output_notebook()

In [80]:
# Load data from CSV
btc_csv = pathlib.Path('../datasets/BTC-USD.csv').absolute()
btc = pd.read_csv(btc_csv, index_col=0, parse_dates=True)

In [83]:
## PLOT: MACD with signal and histograms

In [82]:
# create a StockDataFrame from the BTC dataframe...
sdf = StockDataFrame.retype(btc)

# ...then calculate MACD.
# This will add 5 new columns to the original BTC dataframe:
# 'macd', 'macds' (MACD signal line), 'macdh' (MACD histograms)
# and the 12 and 26 days EMAs of BTC
btc['macd'] = sdf['macd']

In [84]:
# Now plot MACD along with BTC close price

fig_1 = figure(x_axis_type="datetime",  plot_width=800, title='BTC-USD MACD')

# BTC close price line
fig_1.line(btc.index, btc.close, color='darkslategray', legend='BTC close price')

# X-axis
fig_1.line(btc.index, 0, color='black')

# MACD
fig_1.line(btc.index, btc.macd, color='mediumblue', legend='MACD')

# MACD signal
fig_1.line(btc.index, btc.macds, color='darkorange', legend='MACD signal')

# MACD histograms
fig_1.vbar(x=btc.index, # x-coord of bar centers
           bottom=np.zeros(len(btc.index), dtype=int), # y-coords of the bottom edges
           top=btc.macdh,              # height of bars
           width= 12 * 60 * 60 * 1000, # bars width (half a day in milliseconds)
           color="steelblue",
           legend='MACD histograms')

fig_1.legend.location = "top_left"

show(fig_1)

In [ ]:
## PLOT: candlesticks on daily BTC price

In [74]:
# restrict to a few days window
BOX_START = '2017-12-01'
BOX_END = '2018-02-01'
btc_boxed = btc[BOX_START:BOX_END]

In [75]:
# identify bullish and bearish candles 
# and store them in boolean Pandas Series
bullish_candles = btc_boxed.close > btc_boxed.open
bearish_candles = btc_boxed.open > btc_boxed.close

In [ ]:
# plot candlesticks
fig_2 = figure(x_axis_type="datetime", plot_width=800, title='BTC-USD candlesticks')

# A candle is composed by a segment and a bar

# segments identify daily high-low prices      x0, y0, x1, y1
fig_2.segment(btc_boxed.index,  # x-coords of segment start points
              btc_boxed.high,   # y-coords of segment start points
              btc_boxed.index,  # x-coords of segment end points
              btc_boxed.low,    # y-coords of segment end points
              color="darkslateblue")

# bars identify daily open-close prices...

# ...bullish candles: coloured in green
fig_2.vbar(btc_boxed.index[bullish_candles],  # x-coords of candles centers
           12 * 60 * 60 * 1000,               # candlestick width (half day in ms)
           btc_boxed.open[bullish_candles],   # height of candles
           btc_boxed.close[bullish_candles],  # y-coords of bottom edges of candles
           fill_color="yellowgreen")

# ...bearish candles: coloured in green
fig_2.vbar(btc_boxed.index[bearish_candles],  # x-coords of candles centers
           12 * 60 * 60 * 1000,               # candlestick width (half day in ms)
           btc_boxed.open[bearish_candles],   # height of candles
           btc_boxed.close[bearish_candles],  # y-coords of bottom edges of candles
           fill_color="tomato")

# also plot BTC close price
fig_2.line(btc_boxed.index, btc_boxed.close,
           line_width=2,
           color='darkslategray',
           legend='BTC close price')

fig_2.legend.location = "top_left"
fig_2.legend.click_policy = "hide"

show(fig_2)